# Elasticsearch

In [1]:
import logging
import os
import sys

from dotenv import load_dotenv

load_dotenv("../../.env", override=True)

azure_openai_endpoint = os.environ["AZURE_OPENAI_SERVICE_BASEURL"]
azure_openai_key = os.environ["AZURE_OPENAI_SERVICE_TOKEN"]
azure_openai_version = os.environ["AZURE_OPENAI_SERVICE_API_VERSION"]
azure_openai_embedding_deployment_name = os.environ["AZURE_OPENAI_SERVICE_EMBEDDING_DEPLOYMENT"]
azure_openai_embedding_model_name = os.environ["AZURE_OPENAI_SERVICE_EMBEDDING_MODEL_NAME"]
azure_openai_chat_deployment_name = os.environ["AZURE_OPENAI_SERVICE_CHAT_DEPLOYMENT"]
azure_openai_chat_model_name = os.environ["AZURE_OPENAI_SERVICE_CHAT_MODEL_NAME"]

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [2]:
#!pip install llama-index-embeddings-azure-openai
#!pip install llama-index-llms-azure-openai

In [2]:
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.core import Settings


azure_openai_chat_model = AzureOpenAI(
    model=azure_openai_chat_model_name,
    deployment_name=azure_openai_chat_deployment_name,
    api_key=azure_openai_key,
    azure_endpoint=azure_openai_endpoint,
    api_version=azure_openai_version,
)

azure_openai_embedding_model = AzureOpenAIEmbedding(
    model=azure_openai_embedding_model_name,
    deployment_name=azure_openai_embedding_deployment_name,
    api_key=azure_openai_key,
    azure_endpoint=azure_openai_endpoint,
    api_version=azure_openai_version,
)

Settings.llm = azure_openai_chat_model
Settings.embed_model = azure_openai_embedding_model

## Create vector store

In [3]:
from llama_index.vector_stores.elasticsearch import ElasticsearchStore

vector_store = ElasticsearchStore(
    index_name="index_skatteparagraf", es_url="http://localhost:9200/"
)

INFO:elastic_transport.transport:GET http://localhost:9200/ [status:200 duration:0.033s]
GET http://localhost:9200/ [status:200 duration:0.033s]


In [8]:
vector_store = ElasticsearchStore(
    es_url="https://elasticsearch-llm-spring-glitter-3589.fly.dev",
    index_name="index_skatt_para",
    es_user="elastic",
    es_password="abc"
)


INFO:elastic_transport.transport:GET https://elasticsearch-llm-spring-glitter-3589.fly.dev:443/ [status:200 duration:0.183s]
GET https://elasticsearch-llm-spring-glitter-3589.fly.dev:443/ [status:200 duration:0.183s]


## Create pipeline

In [6]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core.node_parser import SentenceSplitter

documents=SimpleDirectoryReader("../../data/skatt-paragraph/").load_data()

In [12]:

pipeline = IngestionPipeline(
    transformations=[
        azure_openai_embedding_model,
    ],
    vector_store=vector_store,
)

node = await pipeline.arun(documents=documents, num_workers=6)

Retrying llama_index.embeddings.openai.base.aget_embeddings in 0.5766072644230048 seconds as it raised RateLimitError: Error code: 429 - {'error': {'code': '429', 'message': 'Requests to the Embeddings_Create Operation under Azure OpenAI API version 2023-12-01-preview have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 50 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.'}}.
Retrying llama_index.embeddings.openai.base.aget_embeddings in 0.8908848633246926 seconds as it raised RateLimitError: Error code: 429 - {'error': {'code': '429', 'message': 'Requests to the Embeddings_Create Operation under Azure OpenAI API version 2023-12-01-preview have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 50 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.'}}.
Retrying llama_index.embedding

BrokenProcessPool: A process in the process pool was terminated abruptly while the future was running or pending.

In [ ]:
len(node)

## Create search engines

In [4]:
from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store,
)

retriever_engine = index.as_retriever(sparse_top_k=5)

In [5]:
query_engine = index.as_query_engine(azure_openai_chat_model, similarity_top_k=5)

In [12]:
from llama_index.core import QueryBundle
query="Hvordan kan jeg få skattefradrag for oppussing av bolig jeg leier ut?"
bundle = QueryBundle(query, embedding=Settings.embed_model.get_query_embedding(query))

INFO:httpx:HTTP Request: POST https://rag-poc-openai.openai.azure.com/openai/deployments/ada-002/embeddings?api-version=2023-12-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://rag-poc-openai.openai.azure.com/openai/deployments/ada-002/embeddings?api-version=2023-12-01-preview "HTTP/1.1 200 OK"


In [13]:
result = query_engine.query(bundle)

INFO:elastic_transport.transport:POST http://localhost:9200/index_skatteparagraf/_search [status:200 duration:0.062s]
POST http://localhost:9200/index_skatteparagraf/_search [status:200 duration:0.062s]


KeyboardInterrupt: 

## Retrieve using vector search

In [6]:
from IPython.display import Markdown, display

def print_markdown(text):
    display(Markdown(text))

def print_text_list(result):
    for text in result:
        print_markdown(text.text)
        print("--"*60)
        print()
        print()

def search_and_display(query):
    result = retriever_engine.retrieve(query)
    print(len(result))
    print_text_list(result)

In [7]:
search_and_display("Skatt")

INFO:httpx:HTTP Request: POST https://rag-poc-openai.openai.azure.com/openai/deployments/ada-002/embeddings?api-version=2023-12-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://rag-poc-openai.openai.azure.com/openai/deployments/ada-002/embeddings?api-version=2023-12-01-preview "HTTP/1.1 200 OK"
INFO:elastic_transport.transport:POST http://localhost:9200/index_skatteparagraf/_search [status:200 duration:0.259s]
POST http://localhost:9200/index_skatteparagraf/_search [status:200 duration:0.259s]
2


U-14-5 Skattesats
Nytt for inntektsåret 2024:
For inntektsåret 2024 er trygdeavgift, mellomsats 7,8 %.
Skattesatsen i ordningen er fastsatt til 25 %, se SSV . § 3-9 første punktum.
Grunnlaget som kildeskattesatsen skal anvendes på skal ikke reduseres med personfradrag. 
Ved fastsetting av skattesatsen på 25 % er det lagt til grunn at skatt yter er pliktig til å betale tryg -
deavgift til Norge etter satsen på 7,9 % for lønnsinntekt.
For personer som er pliktig til å betale trygdeavgift til Norge, men som er under 17 år eller 
over 69 år, er trygdeavgiftssatsen på lønnsinntekt 5,1 % jf. ftrl. § 23-3 annet ledd nr. 1 bokstav c. 
For disse er kildeskattesatsen 22,2 %. Satsen er redusert med 2,8 prosentpoeng, som tilsvarer 
differansen mellom mellomsats og lav sats. Satsen skal framgå av skattekortet.
For personer som er helt eller delvis fritatt medlemskap i folketrygden, skal kildeskattesatsen 
reduseres med gjeldende sats for medlemskapet, jf. SSV § 3-9 annet punktum. Dette innebærer 
for eksempel at en person som ikke er medlem i folketrygden har en kildeskattesats på 17,1 %.
Om hvem som skal betale trygdeavgift, se emnet P-16 Person inntekt – trygdeavgift/pensjons -
poeng/pensjonsbeholdning.

------------------------------------------------------------------------------------------------------------------------




S-22-9.2.4 Selskaper
Det betales en selskapsskatt på 22 % for den del av selskapsoverskuddet som overstiger kr 15 
mill. og som overstiger summen av:
 ̶10 ganger kostnadene til lønn som blir skattlagt ved lønnstrekk, og
 ̶0,20 ganger skattemessig verdi av anlegg, fast eiendom og annen realkapital som er på Sval -
bard ved utgangen av inntektsåret.
For annen inntekt er selskapsskatten 16 % av selskapsoverskuddet.
Se svalbardskattevedtaket § 3 bokstav b og c.

------------------------------------------------------------------------------------------------------------------------




## Query using hybrid search
Denne kommer til å feile fordi vi ikke har Elasticsearch premium

In [ ]:
#response = retriever_hybrid_engine.retrieve("Hva er fartsgrensen i boliggater?")

## Query using HNSW EF Search

In [ ]:
# TODO Investigate how to use

## Query using ES client

In [8]:
from elasticsearch import Elasticsearch

es = Elasticsearch("http://localhost:9200")

In [9]:
# Define the query
query = {
    "query": {
        "match_all": {}
    },
    "size": 5  # Fetch 5 documents
}

# Replace 'your_index' with your actual index name
response = es.search(index="index_skatteparagraf", body=query)

# Print the results
for hit in response['hits']['hits']:
    print(hit['_source'])

INFO:elastic_transport.transport:POST http://localhost:9200/index_skatteparagraf/_search [status:200 duration:0.025s]
POST http://localhost:9200/index_skatteparagraf/_search [status:200 duration:0.025s]
{'embedding': [-0.017635568976402283, -0.0023963626008480787, 0.009938161820173264, -0.03056832030415535, 0.008631055243313313, 0.00274561601690948, -0.02921280264854431, -0.017649400979280472, -0.034468892961740494, -0.00817460473626852, 0.009364141151309013, 0.043708547949790955, -0.016653509810566902, 0.0019329966744408011, -0.01701313816010952, 0.014675560407340527, 0.02101053297519684, -0.014357428066432476, 0.018258001655340195, -0.0005407377029769123, -0.0071717980317771435, 0.013084900565445423, -0.010512182489037514, -0.0008165095932781696, -0.0009500730666331947, -0.010463771410286427, 0.005477400496602058, -0.02196492813527584, 0.0006794881192035973, 0.016224723309278488, 0.001242270227521658, -0.026557093486189842, -0.02890850231051445, -0.013665837235748768, -0.008554980158

In [21]:
response['hits']['total']['value']

10000

### Keyword search

In [12]:
from IPython.display import display, Markdown

# Function to format and display each content
def display_content(contents):
    markdown_content = '\n\n---\n\n'.join(contents)  # Separating contents with horizontal lines
    display(Markdown(markdown_content))

def print_content(contents):
    for content in contents:
        print(content)


def print_es_response(elasticsearch_response):
    # Extract hits
    hits = elasticsearch_response.get('hits', {}).get('hits', [])
    
    # Extract and format content from each hit
    formatted_contents = []
    for hit in hits:
        content = hit.get('fields', {}).get('content', [''])[0]
        formatted_contents.append(content)

    display_content(formatted_contents)
    

In [13]:
def keyword_search(query):
    keyword_result = es.search(
        index="index_skatteparagraf",
        query={
            "match": {
                "content": {
                    "query": query,
                    "boost": 0.9
                }
            }
        },
        size=5,
        fields=["content"],
        _source=False
    )
    
    print()
    print()
    print_es_response(keyword_result)


In [26]:
keyword_search("bolig oppussing")

INFO:elastic_transport.transport:POST http://localhost:9200/index_skatteparagraf/_search [status:200 duration:0.029s]
POST http://localhost:9200/index_skatteparagraf/_search [status:200 duration:0.029s]




B-15-4.3.7 Opphold utenfor hjemmet
At skatt yteren oppholder seg utenfor hjemmet, f.eks. som følge av arbeid, ferie, oppussing mv., 
innvirker ikke på brukstiden så lenge boligen anses som skatt yters faste bolig. Om opphold 

---

B-19-6.4.1 Generelt
Uttak må benyttes til erverv av egen bolig, påkostning av egen bolig eller vedlikehold av egen 
bolig, jf. sktl. § 16-10 første ledd og FSFIN § 16-10-4. Dette omfatter:
 ̶kjøpe-/byggesum for (selve) boligen eller en ideell andel av denne, herunder overtakelse av 
gjeld
 ̶kjøpesum for tomt av passende størrelse til egen bolig, herunder innfestingsavgift for festet 
tomt, eller innløsning av festet tomt
 ̶betaling for nødvendig aksje, andel eller obligasjon
 ̶kostnader til oppussing og oppgradering av egen bolig (påkostning og vedlikehold)
 ̶kostnader ved å utvide/bygge på egen bolig, herunder tilbygg og tilhørende garasje som har tje -
nende funksjon til boligen, se emnet B-17 Bolig – utleie (fritaksbehandling), pkt. B-17-14.1.3
 ̶betaling av renter og avdrag på lån til egen bolig
 ̶betaling av renter og avdrag på felleslån i borettslag hvor skatt yter har sin egen bolig
 ̶utløsning av fraseparert eller fraskilt ektefelle fra egen bolig som har tilhørt ektefellenes fel -
leseie
 ̶dokumentavgift, kostnader til tinglysing og andre ervervskostnader knyttet til erverv nevnt 
ovenfor
For ektefeller anses ektefellenes faste bolig som egen bolig for begge ektefeller dersom begge 
kan anses som reelle eiere. Dersom bare den ene ektefellen står som formell eier av boligen, må 
den andre ektefellen vanligvis anses for å bli medeier dersom BSU-midlene brukes på boligen 
ved f.eks. nedbetaling av ektefellens boliglån, påkostning eller vedlikehold. 

---

F-14-8 Fri bolig
Om fri bolig, se emnet B-12 Bolig – fri bolig.

---

R-6-5.1 Bolig
Verdien av egen bolig, herunder sekundærbolig, fastsettes etter reglene for verdsetting av bolig -
eiendommer, se emnet B-11 Bolig – formue.

---

1752Stikkordregister
Bilgodtgjørelse for besøksreiser  ................   128
Bilgodtgjørelse for daglig reise mellom 
hjem og arbeidssted  ...................................   128
Yrkeskjøring med bilgodtgjørelse som 
følger satsene i satsforskriften  ...................   127
Yrkeskjøring med bilgodtgjørelse som 
ikke følger satsene i satsforskriften  ..........   128
Bilgodtgjørelse ved arbeid for veldedig 
organisasjon/politiske verv, reise hjem-
arbeidssted
Unntak, arbeid for veldedige 
organisasjoner eller politiske verv  ...........   1098
Billedhugger
Kunstnere ...................................................   859
Billetter, velferdstiltak
Forestillinger, idrettsarrangementer mv. ..   1665
Bil, særfradrag for sykdomskostnader
Bil ............................................................   1325
Bil, selskap med deltakerfastsetting
Bil ............................................................   1177
Bioenergianlegg
Fyringsanlegg/varmeanlegg  .......................   740
Biomasse til energiformål, jordbruksfradrag
Spesielt om nettoinntekt fra produksjon av 
biomasse til energiformål, herunder ved  ....  759
Blåskjell
Akvakultur (havbruk) ................................   101
Blind, særfradrag for sykdomskostnader
Synshemming ..........................................   1328
Bøker
Faglitteratur  ................................................   462
Bokføring
Regnskap – foretak med bokføringsplikt  .  1050
Bokføringsplikt, kunstnere
Bokføringsplikt og årsregnskapsplikt  ........   860
Bokføringsplikt, tidfesting
Tidfesting – realisasjonsprinsippet  ..........   1342
Bolig
Bolig – boligselskap mv. og andelshaverne    246
Bolig – eget arbeid  .....................................   256
Bolig – formue  ...........................................   258
Bolig – fri bolig  .........................................   268
Bolig – garasje  ...........................................   274
Bolig – hjemmekontor mv. ........................   276
Bolig – realisasjon  .....................................   281
Bolig – tidsparter (time-share)  ...................   303
Bolig – utleie (fritaksbehandling)  ..............   306
Bolig – utleie (regnskapsbehandling)  ........   323
Boligbyggelag
Samvirkeforetak  .......................................   1156
Bolig, direktebehandling
Bolig – utleie (regnskapsbehandling)  ........   323
Bolig for ansatt
Røkterbolig  ................................................   117
Bolighus på gårdsbruk, inntekt og fradragBolig ..........................................................   736
Bolighus på gårdsbruk, realisasjon
Særlig om våningshuset  .............................   765
Bolig regnskapsbehandling
Bolig – utleie (regnskapsbehandling)  ........   323
Bolig, salg
Bolig – realisasjon  .....................................   281
Boligsameie
Sameie – skattlegging etter brutto-
metoden  ....................................................   1153
Boligselskap
Bolig – boligselskap mv. og andelshaverne    246
Boligselskap, fritaksmetoden
Gevinst og tap på eierandel i boligselskap    641
Boligselskap, ikke eget skattesubjekt
Selskaper og sammenslutninger som 
skattlegges som eget skattesubjekt  ..............   66
Boligselskap/leid eiendom mot innskudd, 
beregnet personinntekt
Særregel for leid eiendom mot innskudd  ...  438
Bolig, selskap med deltakerfastsetting
Bolig ........................................................   1177
Bolig, selvstendig/uselvstendig bolig, definisjon
Selvstendig/uselvstendig bolig  ..................   345
Boligsparing for ungdom
Boligsparing for ungdom (BSU) ...............   330
Bompenger for bil, fradrag på yrkesreise
Parkeringsavgift, bompenger, 
piggdekkavgift og fergekostnad .................   225
Bompenger på arbeidsreise
Fradrag i tillegg til avstandsfradraget, 
faktiske kostnader til fergereise og 
bompassering ...........................................   1094
Borettslagsleiligheter
Bolig – boligselskap mv. og andelshaverne    246
Bolig – realisasjon  .....................................   281
Børshandlede produkter
Finansielle instrumenter – ETP (Exchange 
Traded Products) ........................................   476
Børsintroduksjon
Kostnader ved stiftelse av selskap, 
nytegning av aksjekapital og 
børsintroduksjon ..........................................   73
Bortfeste
Rettigheter i fast eiendom  ........................   1144
Bortleie
Bolig – utleie (fritaksbehandling)  ..............   306
Bolig – utleie (regnskapsbehandling)  ........   323
Bosatt
Bosted – skattemessig bosted  ....................   338
Utland – skattemessig bosted  ...................   1553
Bosted
Bosted – skattemessig bosted  ....................   338
Utland – skattemessig bosted  ...................   1553
Bostedsbekreftelse i forhold til utlandet

### Vector search

In [22]:
def vector_search(query):
    query_embedding = azure_openai_embedding_model.get_text_embedding(query)

    response_vector = es.search(
        index="index_skatteparagraf",
        query={
            "knn": {
                "field": "embedding",  # Ensure this matches the name of your dense_vector field
                "query_vector": query_embedding,
                "num_candidates": 50,  # This is optional but can help with performance/accuracy trade-offs
                "boost": 0.1,
            }
        },
        size=10,
        fields=["content"],
        _source=False,
    )
    print()
    print()
    print_es_response(response_vector)

In [23]:
vector_search("Hvordan kan jeg få fradrag på skatten for oppussing av bolig?")

INFO:httpx:HTTP Request: POST https://rag-poc-openai.openai.azure.com/openai/deployments/ada-002/embeddings?api-version=2023-12-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://rag-poc-openai.openai.azure.com/openai/deployments/ada-002/embeddings?api-version=2023-12-01-preview "HTTP/1.1 200 OK"
INFO:elastic_transport.transport:POST http://localhost:9200/index_skatteparagraf/_search [status:200 duration:0.028s]
POST http://localhost:9200/index_skatteparagraf/_search [status:200 duration:0.028s]




B-19-2.1 Hvem kan kreve fradrag
Personlige skatt ytere som er skattepliktig som bosatt i Norge og som ikke eier bolig per 31. 
desember i inntektsåret, kan til og med det inntektsåret de fyller 33 år, kreve fradrag i skatt etter 
reglene for boligsparing for ungdom (BSU), jf. sktl. § 16-10 første ledd.
Personer som ikke er skattepliktig som bosatt i Norge kan som hovedregel ikke kreve slikt 
fradrag i skatten. Dette gjelder også for slike personer som under midlertidig opphold i Norge er 
skattepliktig hit etter sktl. § 2-3 første ledd bokstav d og § 2-3 annet ledd, se emnet U-17 Utland 
– lønnsinntekter og pensjonsinntekter mv., pkt. U-17-1.2. Personer bosatt i annen EØS-stat med 
begrenset skatteplikt til Norge etter sktl. §  2-3 kan likevel på visse vilkår kreve slikt fradrag, 
se sktl. § 6-71 og emnet U-17 Utland – lønnsinntekter og pensjonsinntekter mv., pkt. U-17-14. 
Personer som er bosatt i utlandet og arbeider i norsk utenrikstjeneste, jf. sktl. § 2-1 åttende ledd, 
gis i praksis BSU-fradrag selv om de bare er begrenset skattepliktig til Norge for lønnsinntekten.

---

B-14-2.2.3 Hjemmekontor i leid bolig
Kreves fradrag for faktisk påløpte kostnader ved å holde hjemmekontor i leid bolig, må samlede 
kostnader til husleie mv. fordeles skjønnsmessig på henholdsvis boligdelen og hjemmekontoret. 
Fordelingen foretas forholdsmessig ut fra leieverdien. Alternativt kan arealfordelingen legges til 
grunn. Det kan ikke kreves fradrag for skatt  yterens eget eller ektefelles arbeid med renhold av 
arbeidsrom/hjemmekontor.

---

B-14-2.2.1 Hjemmekontor mv. i fritaksbehandlet bolig
I de tilfeller skatt yter har hjemmekontor i fritaksbehandlet bolig, herunder andelshavers leilighet 
i boligselskap, gis et standardfradrag med kr 2 050 for hjemmekontor mv., jf. takseringsreglene 
§ 1-3-26 første ledd. Fradraget skal ikke reduseres selv om skatt yter har bodd i huset bare en del 
av året eller kun brukt hjemmekontoret en del av året.
Istedenfor standardfradrag kan eieren kreve fradrag for faktiske kostnader knyttet til ervervs -
delen av boligen (hjemmekontor mv.), jf. takseringsreglene § 1-3-26 annet ledd. Kostnadene må 
sannsynliggjøres, herunder må det sannsynliggjøres at de faktiske kostnadene som det kreves 
fradrag for, kun refererer seg til hjemmekontoret og ikke gjelder boligdelen av huset. Kostnader 
som ikke kan henføres til spesielle deler av eiendommen, som for eksempel kostnader til ytre 
vedlikehold, forsikring og kommunale avgifter, fordeles i forhold til leieverdien for boligdelen 
og ervervsdelen (hjemmekontor mv.). Hvis boligdelen og ervervsdelen (hjemmekontoret mv.) 
må anses å være likeverdige og leieverdien av ervervsdelen er vanskelig å fastslå, kan kostnader 
som ikke kan henføres til spesielle deler av eiendommen, fordeles forholdsmessig ut fra en are -
alfordeling mellom boligdelen og ervervsdelen. Det gis ikke fradrag for skatt  yterens eget eller 
ektefelles arbeid med renhold av arbeidsrom/hjemmekontor.

---

B-12-4.3 Fri bolig for andre
Det forekommer at en skatt  yter stiller bolig vederlagsfritt til disposisjon for andre, mens han fort -
satt bærer kostnadene vedrørende boligen. Forutsetningen for å få fradrag for disse kostnadene er 
at boligen regnskapsbehandles og at kostnadene kan anses å være pådratt for å erverve, sikre eller 
vedlikeholde inntekt som er skattepliktig for eieren. Slik inntekt må kunne påregnes for eieren 
i løpet av overskuelig fremtid. Skattepliktig bruk etter sktl. §  5-20 tredje ledd om vederlagsfri 
bruk gir ikke grunnlag for fradrag for eieren, se eksempel i URD 29. september 2004 (Dalane 
tingrett) i Utv. 2005/294.  Om fradragsrett for vedlikehold knyttet til en tidligere skattepliktig 
utleieaktivitet, se emnet B-18 Bolig – utleie (regnskapsbehandling), pkt. B-18-3.6. Om fradrags -
rett ved overgang fra fritaksbehandling til regnskapsbehandling, se emnet B-18 Bolig – utleie 
(regnskapsbehandling), pkt. B-18-3.11. Er ytelsen gitt som føderåd, er ytelsen fradragsberettiget 
etter sktl. § 6-52, se emnet F-36 Føderåd
Hvis deler av boligen leies ut i tillegg til den vederlagsfrie bruk, vil leieinntektene være skat -
tepliktige etter de vanlige reglene og kostnadene knyttet til utleien vil være fradragsberettiget. 
Den vederlagsfrie bruken medfører ikke at boligen skal fritaksbehandles.

---

Boligsparing for ungdom (BSU)
337
av sparebeløpet, mens fradrag i skatt gis med bare 10 % fra og med inntektsåret 2023. Tillegget 
i skatt skal tilsvare det reelle fradraget som ble gitt tidligere år. 
Har skatt yter ikke fått utnyttet skattefradraget fullt ut, fordi fastsatt skatt har vært lavere enn 
skattefradraget, er det det faktiske skattefradraget som utgjør tillegget.
Skatt yteren gis tillegg i skatt for hele det tidligere skattefradraget selv om deler av skattefra -
draget i sin tid ble gitt til ektefellen som følge av at skatt  yteren selv ikke kunne nyttiggjøre seg 
fradraget, se pkt. B-19-3.2.
Hvis en skatt yter som har fått overdratt BSU-midler fra ektefelle ved skilsmisse bryter vilkå -
rene, vil denne få tillegg i skatt både på grunnlag av det overførte beløp og egen sparing.
Har skatt yter brutt vilkårene, men brukt deler av sparemidlene i samsvar med vilkårene, skal 
det bare gis tillegg i skatt for den delen av sparemidlene som ikke er brukt i samsvar med vilkå -
rene, herunder innskutte beløp som ikke er tatt ut. Ved beregningen av tillegget i skatt anses beløp 
som er brukt til erverv av egen bolig først å komme fra innbetalt sparebeløp, deretter akkumulerte 
renter påløpt til og med det året skatt yteren fylte 33 år.
Eksempel
En skatt yter har en sparekonto med innestående kr 130 000, hvorav kr 100 000 i innbetalte 
sparemidler og kr 30 000 er renter påløpt før fylte 33 år. Skatt yteren tar ut kr 80 000 og 
bruker kr 65 000 til erverv av egen bolig innen utløpet av uttaksåret, mens resten av uttaket, 
kr 15 000, brukes til kjøp av innbo. Det står kr 50 000 igjen på kontoen.
Fordi ikke hele uttaket er brukt til erverv av bolig mv., er vilkårene brutt. Skatt yteren 
skal ha et tillegg i skatt. Han har brukt kr 65 000 etter vilkårene. Av det totalt innbetalte 
sparebeløp, vil det da være kr 100 000 – kr 65 000 = kr 35 000 som ikke er benyttet etter 
vilkårene. Tillegget i skatt blir da 20 % av kr 35 000, dvs. kr 7 000. Satsen på 20 % benyttes 
for hele beløpet siden det ble gitt fradrag i skatt med 20 % av sparebeløpet i alle de tidligere 
årene.

---

I-2-2.2.2 Utleie av bolig/fritidsbolig med innbo og løsøre
Ved utleie av møblert fritaksbehandlet bolig, ses det bort fra overskudd/underskudd for møbel -
utleien.
Ved utleie av fritaksbehandlet fritidseiendom blir 85 % av leien over kr 10 000 skattlagt. 
Skatt yteren skal ikke gis særskilt fradrag for kostnader vedrørende møblene. Tilsvarende gjelder 
hvor leien inkluderer rett til å bruke annet løsøre som naturlig hører til fritidsboligen, f.eks. en 
fritidsbåt. Det samme gjelder ved utleie av egen bolig som faller inn under sjablongregelen om 
korttidsutleie av egen bolig, se emnet B-17 Bolig – utleie (fritaksbehandling), pkt. B-17-4.3.
I regnskapsbehandlet bolig og fritidseiendom tas hele leien til inntekt og det gis fradrag for 
kostnader vedrørende møblene/løsøret etter de vanlige regler for driftsmidler. For nyinnkjøpt 
innbo/løsøre legges faktisk kostpris til grunn. For innbo/løsøre brukt av eieren, settes inngangs -
verdien til skjønnsmessig salgsverdi ved begynnelse av utleien. Dersom skattepliktig utleie av 
fullt møblert bolig/fritidsbolig antas å være kortvarig, er det i praksis akseptert at en forenklet, 
skjønnsbasert metode anvendes. Dette gjelder tilfeller hvor eieren selv bruker innboet privat før 
utleieperioden og det er sannsynlig at han også skal benytte innboet etter utleieperioden, f.eks. ved 
utleie av egen bolig ved midlertidig fravær. I slike tilfeller kan fradraget i praksis settes til 15 % 
av brutto leieinntekt (ev. inntekt fra korttidsutleie av egen bolig holdes utenfor). Denne regelen 
bør ikke brukes når utleieperioden overstiger 3 år.

---

S-22-2.5.15 Boligsparing for ungdom, skattefradrag
Personer som er bosatt på Svalbard, og som fortsatt er skattepliktig som bosatt på fastlandet, har 
krav på skattefradrag for sparebeløp i boligspareordningen for ungdom (BSU) etter sktl. § 16-10 
i fastsatt skatt på fastlandet. Dette gjelder selv om vedkommende samtidig er skattepliktig som 
bosatt på Svalbard. Personer som ikke er skattepliktig som bosatt på fastlandet, har ikke krav på 
slikt fradrag ved midlertidig arbeidsopphold på fastlandet.

---

U-17-13.1.1 Generelt
Personer som er skattepliktig som bosatt i Norge og skattepliktig hit for lønnsinntekt opptjent i 
utlandet, har rett til fradrag på vanlig måte for kostnader knyttet til denne inntekten.
Har skatt yter vært bosatt i Norge en del av året, er det fradragsrett for kostnader som knytter 
seg til inntekt som er skattepliktig til Norge, jf. sktl. § 6-92 første punktum. Dette gjelder f.eks. 
reisekostnader, flyttekostnader mv. Tilsvarende gjelder fagforeningskontingenter og pensjonspre -
mier. Om minstefradrag, se emnet M-6 Minstefradrag.
Personer som blir skattemessig bosatt i Norge i løpet av inntektsåret og er i kildeskatteordnin -
gen for lønnsinntekt, har ikke rett til fradrag i inntekt som skattlegges i ordningen. Om kildeskatt 
på lønn, se emnet U-14 Utland – kildeskatt på lønn for utenlandske arbeidstakere.
Om grensegjengere, se emnet U-11 Utland – grensegjengere.

---

Bolig – utleie (regnskapsbehandling)
329
Fradrag for vedlikeholdskostnader skal beregnes slik:
Antall år med fritaksbehandling 
i siste femårsperiode:Fradrag for vedlikeholdskostnader gis med:
5 år kr 10 000 + 50 % av det som overstiger kr 10 000
4 år kr 10 000 + 60 % av det som overstiger kr 10 000
3 år kr 10 000 + 70 % av det som overstiger kr 10 000
2 år kr 10 000 + 80 % av det som overstiger kr 10 000
1 år kr 10 000 + 90 % av det som overstiger kr 10 000
0 år Fullt fradrag
Eksempel 6
En eiendom består av en familieleilighet med en årlig utleieverdi på kr 70 000 og en hybel 
med en årlig utleieverdi på kr 30  000. Familieleiligheten blir i løpet av året benyttet 3 
måneder av eieren mens den de resterende 9 månedene blir utleid. Hybelen er derimot utleid 
hele året.
Eieren pådrar seg i løpet av året vedlikeholdskostnader på familieleiligheten med 
kr 100 000. Som følge av at eiendommen blir regnskapsbehandlet og egen bruk som bolig 
ikke er skattepliktig, vil kr 75 000 (100 000 x 9/12) utgjøre grunnlaget før reduksjon etter 
sktl. § 7-10.
Eksempel 7
Samme eiendom som i eksemplet ovenfor, men her pådrar eieren seg i løpet av året vedli -
keholdskostnader med kr 100 000 som ikke kan henføres til en spesiell del av eiendommen 
(felleskostnad). Som følge av at eiendommen regnskapsbehandles og egen bruk som bolig 
ikke er skattepliktig, vil kr 82 500 (70 000 x 9/12 + 30 000 x 12/12) utgjøre grunnlaget før 
reduksjon etter sktl. § 7-10.
I noen tilfeller kan årsaken til at eieren ikke har hatt regnskapsbehandlet leieinntekt i ett eller flere 
av de fem foregående årene, være at en nær slektning av eieren har bodd vederlagsfritt i boligen 
mot å dekke løpende kostnader, og derved kommet inn under skattefritaksregelen for egen bruk 
i sktl. § 7-1, se emnet B-17 Bolig – utleie (fritaksbehandling), pkt. B-17-8. Eierens fradrag for 
vedlikeholdskostnader skal også i disse tilfellene reduseres på tilsvarende måte som om han hadde 
bodd i boligen selv i samme tidsrom.

---

B-10-2.1 Generelt
Unntatt fra skatteplikt er den fordelen som oppstår ved at skatt yteren og dennes husstand utfører 
fritidsarbeid på egen bolig og fritidseiendom, se sktl. § 5-15 første ledd bokstav j nr. 6. Skattefri -
taket gjelder både arbeid på nybygg og arbeid på eksisterende bygg, også hvor bygget tidligere 
har vært leid ut. Det samme gjelder fritidsarbeid på bygg tilhørende boligbyggelag til opptjening 
av eget innskudd i laget. Skattefritaket omfatter alle typer fritidsinnsats på egen bolig, også ved -
likeholdsarbeid, se FIN 19. september 2001 i Utv. 2001/1644.
Fritaket kan også omfatte arbeid med å klargjøre tomt for bygging, f.eks. opparbeidelse av 
vei, vann og kloakk. Verdien av arbeid med å anlegge en vanlig hage omfattes også av fritaket.
Fritaket gjelder alle typer bolig, herunder bolig på gårdsbruk. Fritaket gjelder også for verdien 
av arbeid på egen boligdel i bygg som sett under ett er avskrivbart, idet egen boligdel skal være 
trukket ut av avskrivningsgrunnlaget.

### Hybrid søk

In [25]:
def hybrid_search(query, keyword):

    query_hybrid_embedding = azure_openai_embedding_model.get_text_embedding(query)
    query_hybrid_keyword = keyword
    
    response_hybrid = es.search(
        index="index_skatteparagraf",
        query={
            "match": {"content": {"query": query_hybrid_keyword, "boost": 0.9}},
        },
        knn={
            "field": "embedding",
            "query_vector": query_hybrid_embedding,
            "k": 5,
            "num_candidates": 50,
            "boost": 0.1,
        },
        fields=["content"],
        _source=False,
    )

    print()
    print()
    print_es_response(response_hybrid)

In [26]:
hybrid_search("Hvordan kan jeg få fradrag på skatten for oppussing av bolig?", "§ 6-48 § 6-50 § 6-31 § 6-40")

INFO:httpx:HTTP Request: POST https://rag-poc-openai.openai.azure.com/openai/deployments/ada-002/embeddings?api-version=2023-12-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://rag-poc-openai.openai.azure.com/openai/deployments/ada-002/embeddings?api-version=2023-12-01-preview "HTTP/1.1 200 OK"
INFO:elastic_transport.transport:POST http://localhost:9200/index_skatteparagraf/_search [status:200 duration:0.053s]
POST http://localhost:9200/index_skatteparagraf/_search [status:200 duration:0.053s]




Lov og forskriftsregister 
1739
§ 5-13  232
§ 5-13 første ledd  937
§ 5-14 første ledd  11
§ 5-14 tredje ledd  488
§ 5-15  1689
§ 5-15 første ledd  1415
§ 5-15 første ledd bokstav a  1221, 1222
§ 5-15 første ledd bokstav b  1400
§ 5-15 første ledd bokstav d  631
§ 5-15 første ledd bokstav e  1213, 1284
§ 5-15 første ledd bokstav e nr. 1  937
§ 5-15 første ledd okstaven r.1  531
§ 5-15 første ledd bokstav f  631
§ 5-15 første ledd bokstav g  701
§ 5-15 første ledd bokstav h  1222
§ 5-15 første ledd bokstav i  1356, 1357
§ 5-15 første ledd bokstav j  1010, 1287
§ 5-15 første ledd bokstav j nr. 1  859
§ 5-15 første ledd bokstav j nr. 4  603
§ 5-15 første ledd j nr. 6  256
§ 5-15 første ledd bokstav k  1221, 1222
§ 5-15 første ledd bokstav l  268, 631
§ 5-15 første ledd bokstav m  1222
§ 5-15 første ledd bokstav n  268
§ 5-15 annet ledd  1430, 1431
§ 5-15 annet ledd fjerde punktum  1244
§ 5-20 første ledd bokstav b  1101, 1668
§ 5-20 første ledd bokstav c  950
§ 5-20 første ledd bokstav d  948, 950
§ 5-20 annet ledd  592
§ 5-20 tredje ledd  232, 676, 679, 1144
§ 5-21 første ledd  709
§ 5-21 annet ledd  592
§ 5-22  1101
§ 5-30  1691
§ 5-30 første ledd  19, 398
§ 5-30 annet ledd  398, 474, 486, 502, 506
§ 5-30 fjerde ledd  965, 968, 972
§ 5-31 bokstav a  1400
§ 5-31 bokstav c  613
§ 5-40  965, 972, 977, 991
§ 5-40 første ledd  186
§ 5-40 tredje ledd  960, 962
§ 5-40 fjerde ledd  960, 962
§ 5-41  597
§ 5-42  1414, 1415
§ 5-42 bokstav a  679, 1287
§ 5-42 bokstav b  957, 960, 962
§ 5-42 bokstav c  679
§ 5-42 c  1287
§ 5-42 bokstav d  789
§ 5-42 bokstav e  1400
§ 5-43 første ledd bokstav i og j  1239
§ 5-43 første ledd  1400
§ 5-43 første ledd bokstav d  1415§ 5-43 første ledd bokstav e  1415
§ 5-43 første ledd bokstav g  1222, 1415
§ 5-43 annet ledd  1415
§ 5-50  1357
§ 5-50 tredje ledd  676, 679
§ 5-60  1101
kap. 6  796, 798
§ 6-1  782, 793, 888, 1074, 1288, 1289, 1291, 
1292, 1408, 1409, 1430, 1431, 1688, 1689
§ 6-1 første ledd  977
§ 6-2  1329
§ 6-2 første ledd  19, 398, 474, 486, 502, 506
§ 6-2 annet ledd  552
§ 6-3  1419, 1420
§ 6-3 annet ledd  550
§ 6-3 tredje ledd  550
§ 6-3 fjerde ledd  1312, 1313
§ 6-10  352, 360
§ 6-10 tredje ledd  389
§ 6-11  1652, 1653
§ 6-12  222, 232
§ 6-13  864, 888, 892, 901, 1074, 1288, 1289
§ 6-13 første ledd  925
§ 6-14  421
§ 6-15  1567, 1568
§ 6-16  186
§ 6-17  1144
§§ 6-18 til 6-20  793
§ 6-19 annet ledd  1062
§ 6-21  1098, 1099
§ 6-22  1418
§ 6-24  1409
§ 6-25  1341, 1342
§§ 6-30 til 6-32  929
§ 6-30  546, 778, 781
§ 6-30 første ledd  901, 918, 920
§ 6-30 annet ledd  603
§ 6-31 første ledd d  204
§ 6-31 annet ledd  859
§ 6-32 annet ledd  1239
§ 6-40  1127, 1681
§ 6-40 åttende ledd  1342
§ 6-41  676, 679, 1107
§ 6-42  679
§ 6-43  1431
§ 6-44  892, 1074, 1088
§ 6-45  957
§ 6-46  957, 965, 968, 972, 980
§ 6-47  957, 965, 968, 972, 980
§ 6-47 første ledd bokstav c  960, 962
§ 6-47 første ledd bokstav e  603
§ 6-47 første ledd bokstav f  603
§ 6-47 annet ledd  592, 597
§ 6-48  564
§ 6-49  199

---

Lov og forskriftsregister 
1745
§ 5-22  1101
§ 5-41  958, 960
§ 5-43  960, 968
§ 6-2  1328, 1329
§ 6-12  232
§ 6-13  892
§ 6-13 B  925
§ 6-31  706, 929
§ 6-40  1127
§ 6-41-1 til § 6-41-3  1107
§ 6-44  1074, 1088
§ 6-44-5  892
§ 6-47  960
§ 6-47-11  570
§ 6-50  679
§ 6-51-1  796, 798
§ 6-60  531
§ 6-61  1213
§ 6-72-1  965
§ 6-83  1314
§ 7-3  246
§ 8-1 B  728
§ 8-1-1  952
§§ 8-1-10 til 8-1-13  756
§ 8-2  1267
§ 9-3  713
§ 9-8  713, 761, 1403, 1404
§ 9-10  477
§ 9-14  1598
§ 10-11  1156
§ 10-11 og § 10-12  40
§ 10-12-1  1202
§ 10-32  19
§ 10-42-1  1202
§§ 10-42-2 til 10-42-4  1202
§ 10-44-1  1188, 1194, 1195
§ 10-48  1202
§ 10-48-1  1195
§ 10-68  19
§ 10-70  1473
§ 11-20  1713
§ 11-21  86
kap. 12  429
kap. 14  1336
§ 14-3  1337
§ 14-3 B  631
§ 14-3-1  459
§ 14-5  1342
§ 14-5 A  1649
§ 14-5 B  552
§ 14-5 C  1641
§ 14-5 E  552
§ 14-5-2  859
§ 14-41  360
§ 14-62  1456§ 16-28  1568
§ 16-40  613
Forskrift av 22. november 1999 nr. 1160 
fastsatt av Skattedirektoratet til utfylling og 
gjennomføring av skatteloven av 26. mars 
1999 nr. 14 (FSSD)
§ 5-12  937
§ 5-13  232
§ 6-13  864, 901, 923, 925
§ 6-21  1139
§ 6-21-2 tredje ledd  781
§ 6-21-3  1661
§ 6-21-5  1099
kap. 7  306
§ 7-3  246
§ 7-11  246
§ 10-50  1156
§ 12-2  989, 991
§ 12-21  1691
§ 12-22  1691
§ 12-2-2 annet ledd  429
§ 12-2-7 tredje ledd  429
§ 16-30  1568
Forskrift 19. desember 2014 nr. 1890 om 
endring i forskrift til utfylling og gjen -
nomføring av skatteloven av 26. mars 1999 
nr. 14 II  1164
Lov 24. mars 2000 nr. 16 om foretakspensjon 
(foretakspensjonsloven)
Forskrift 1. desember 2000 nr. 1212 til lov 
24. mars 2000 nr. 16 om foretakspensjon  
968
Forskrift 22. desember 2000 nr. 1412 om 
overgangsregler for private tjenestepen -
sjonsordninger  968
Lov 24. november 2000 nr. 81 om 
innskuddspensjon i arbeidsforhold 
(innskuddspensjonsloven)
Forskrift 21. november 1989 nr. 1170 om 
forvaltning og bruk av midler i premiefond, 
pensjonsreguleringsfond og innskuddsfond  
972
Forskrift 22. desember 2000 nr. 1413 til lov 
24. november 2000 nr. 81 om innskudds-
pensjon i arbeidsforhold  972
Lov 19. november 2004 nr. 73 om bokføring 
(bokføringsloven)
Forskrift 1. desember 2004 nr. 1558 om bok-
føring (bokføringsforskriften)  1050, 1626, 
1649
§ 4-2  1641

---

D-1 Driftsmiddel – allment om 
fradrag for inngangsverdi
 ̶Sktl. § 6-10, § 14-40, § 14-43, § 14-50 og § 14-52

---

B-2 Barnepass
 ̶Sktl. § 6-31 første ledd bokstav d
 ̶Sktfvl. § 7-2 nr. 1 bokstav j
 ̶Sktfvl. § 7-10 bokstav b
 ̶Ftrl. § 23-3 annet ledd nr. 2 bokstav d
 ̶Takseringsreglene § 1-3-40
 ̶Satsforskriften § 11
 ̶FSSD § 12-2-6

---

D-4 Driftsmiddel – immaterielt 
(ikke-fysisk) driftsmiddel
 ̶Sktl. § 4-2 første ledd bokstav f, § 6-10 første, annet og tredje ledd, § 14-30 til § 14-48, § 14-50, 
§ 14-52 og § 14-53
 ̶Rl. § 5-7, 6-2 A I og § 7-14 tredje ledd
 ̶Norsk RegnskapsStandard 19 Immaterielle eiendeler (desember 2012)

---

A-17 Arvelater/etterlatte/dødsbo
 ̶Sktl. § 2-2 første ledd bokstav h nr. 2, § 2-15, § 5-40 første ledd,§ 5-43 første ledd bokstav d 
nr. 5, § 5-50 tredje ledd, § 6-16, § 9-2 tredje ledd bokstav b og c, § 9-7, § 14-48 første ledd og 
§ 15-4 femte ledd
 ̶Lov 14. juni 2019 nr. 21 om arv og dødsboskifte (arveloven) § 11, § 14, § 22, § 32, § 36, § 118, 
§ 160, § 163, kap. 3 og kap. 17
 ̶Lov 4. juli 1991 nr. 47 om ekteskap (ekteskapsloven) § 31, § 42, § 58 første ledd og § 59
 ̶Lov 13. desember 2013 nr. 110 om oppheving av lov 19. juni 1964 nr. 14 om arveavgift (arve -
avgiftsloven)
 ̶FSFIN § 9-8

---

M-6 Minstefradrag 
 ̶Sktl. §§ 6-30 til 6-32
 ̶FSFIN § 6-31
 ̶SSV § 6-1

---

M-6-5 Kostnader som ikke inngår i minstefradraget 
Følgende kostnader omfattes ikke av minstefradraget og kommer derfor særskilt til fradrag der -
som vilkårene for fradrag foreligger:
 ̶daglige reiser mellom hjem og fast arbeidssted (arbeidsreiser), jf. sktl. § 6-44
 ̶merkostnader til kost og losji ved arbeidsopphold og på tjenestereiser med overnatting utenfor 
hjemmet, samt underskudd på godtgjørelse til dekning av slike kostnader, jf. sktl. § 6-13
 ̶merkostnader til kost og losji på reiser med overnatting ved flytting for å overta ny stilling og 
i forbindelse med konferanse om stilling, jf. sktl. § 6-13
 ̶besøksreiser til hjemmet, jf. sktl. § 6-13 første ledd siste punktum og § 6-44
 ̶premie og tilskudd til privat og offentlig pensjonsordning i arbeidsforhold, jf. sktl. § 6-47 
første ledd bokstav a og b
 ̶premie, innskudd og kostnader knyttet til administrasjon av individuell pensjonsavtale etter 
lov 27. juni 2008 nr. 62 om individuell pensjonsordning, jf. sktl. § 6-47 første ledd bokstav c
 ̶premie, innskudd og kostnader knyttet til administrasjon av avtaler om individuell sparing til 
pensjon, jf. sktl. § 6-47 første ledd bokstav d
 ̶fradragsberettiget fagforeningskontingent, jf. sktl. § 6-20
 ̶foreldrefradrag, jf. sktl. § 6-48
 ̶føderådsytelser (kår), jf. sktl. § 6-52
 ̶underskudd i virksomhet og ved drift av fast eiendom, jf. sktl. § 6-3
 ̶renter av gjeld, jf. sktl. § 6-40
 ̶gaver til visse frivillige organisasjoner, jf. sktl. § 6-50, se emnet G-2 Gaver, tilskudd og under -
støttelser utenfor arbeidsforhold
 ̶tilskudd til vitenskapelig forskning og yrkesopplæring, jf. sktl. § 6-42, se emnet G-2 Gaver, 
tilskudd og understøttelser utenfor arbeidsforhold
Oppregningen er ikke uttømmende.

---

1740 Lov og forskriftsregister
§ 6-50  679
§ 6-51  564
§ 6-52  752, 975, 977
§ 6-60  531
§ 6-61  1213
§ 6-70  892, 901, 920, 1213
§ 6-72  957
§ 6-80  1312
§ 6-81  1313
§ 6-82  1313
§ 6-85  1313
§ 6-90  550
§ 6-90 første ledd bokstav b  1420
§ 6-90 annet ledd  1062
§ 6-90 tredje ledd  410
§ 6-90 fjerde ledd  1419, 1420
kap. 7  274, 303, 306, 323
§ 7-1  728
§ 7-1 første ledd bokstav b  246
§ 7-3  65, 246
§ 7-10  728
§ 8-1 første ledd bokstav a  952, 1061, 1062
§ 8-1 første ledd bokstav b  531
§ 8-1 første ledd bokstav c  728
§ 8-1 første ledd bokstav d  728
§ 8-1 annet ledd  952
§ 8-1 tredje ledd  728
§ 8-1 fjerde ledd  725, 728
§ 8-1 femte ledd  728, 756
§ 8-1 sjette ledd  1062
§ 8-1 sjuende ledd  1402
§ 8-1 åttende ledd  1062, 1402
§ 8-1 niende ledd  728
§ 8-1 tiende ledd  728
§ 8-2  1267
§ 8-3  1062
§ 8-6  452, 531
§§ 8-10 til 8-20  1020
§ 8-18 tredje ledd  74
kap. 9  398
§ 9-2  1011, 1329
§ 9-2 første ledd bokstav a  452
§ 9-2 første ledd bokstav c  207
§ 9-2 første ledd bokstav e  1144
§ 9-2 første ledd h  19
§ 9-2 annet ledd  1144
§ 9-2 tredje ledd bokstav b  186
§ 9-2 tredje ledd bokstav c  186
§ 9-2 tredje ledd bokstav h  789
§ 9-3 første ledd bokstav a  649, 707, 709
§ 9-3 første ledd bokstav b  1641
§ 9-3 første ledd bokstav c  1667, 1668
§ 9-3 sjette ledd  761
§ 9-3 sjuende ledd  806, 809, 848
§ 9-3 åttende ledd  1404§ 9-3 åttende ledd bokstav b  1144
§ 9-3 åttende ledd bokstav c  1144
§ 9-3 åttende ledd bokstav d  1144
§ 9-3 niende ledd  761
§ 9-4  486, 502, 506, 552, 1328, 1329, 1640, 
1641
§ 9-4 første ledd  709, 761, 1667, 1668
§ 9-6  710, 713
§ 9-7  710, 713, 1061, 1062
§ 9-8  713
§ 9-9  1641
§ 9-10  477, 488
§ 9-11  1681
§ 9-12  1681
§ 9-13  761
§ 9-14  663, 1597, 1598
§ 10-1  65
§ 10-1 første ledd  1156
§ 10-1 annet ledd  246
§ 10-1 tredje ledd  1673
§§ 10-2 til 10-4  74
§ 10-5  1673
§§ 10-10 til 10-13  40
§ 10-10  1681
§ 10-10 tredje ledd  1673
§ 10-11  268
§ 10-11 første ledd  1673
§ 10-11 annet ledd  871, 876, 882
§ 10-11 tredje ledd  1681
§ 10-11 sjette ledd  1156
§ 10-11 åttende ledd  1156
§ 10-13  636
§ 10-13 første ledd  1681
§§ 10-30 til 10-37  19
§ 10-30 første ledd  1673
§ 10-32 annet ledd  713
§ 10-33  163
§ 10-36 fjerde ledd  636
§ 10-37 tredje ledd  1595
§§ 10-40 til 10-48  1164
§ 10-40  636, 1170
§ 10-40 annet ledd  809, 848
§ 10-41  636, 1170
§ 10-42  871, 876, 1201, 1202
§ 10-44  1188, 1194, 1195
§ 10-45  1184, 1187, 1188
§ 10-46  163
§ 10-48  1202, 1712, 1713
§ 10-50  1156
§ 10-70  1473
§ 10-71  1594, 1595
§§ 11-1 til 11-3  650, 663
§ 11-1  510, 524
§§ 11-4 til 11-10  510
§ 11-4  524

---

G-2 Gaver, tilskudd og understøttelser 
utenfor arbeidsforhold
 ̶Sktl. § 5-20 tredje ledd, § 5-42 bokstav b og bokstav c, § 5-50 tredje ledd,§ 6-42, § 6-50, § 6-52 
og § 9-7
 ̶Sktfvl. § 7-10 bokstav a og bokstav c
 ̶Skatteforvaltningsforskriften § 7-10 A og C
 ̶FSFIN § 6-50

## Search options

### Keyword search

Search on specific matches on keywords:

`keyword_search("Keywords...")`

### Vector search

Search on input text intention match:

`vector_search("Input sentence...")`

### Hybrid search

Search on a combination of keyword match and sentence intent match:

`hybrid_search("Sentence intent...", "keywords...")`

## Answer using chatGPT

In [104]:
from llama_index.core.llms import ChatMessage

def prepare_prompt(query, responses):
    context = "\n\n".join(responses)  # Join documents with double newlines for clarity
    prompt = f"Du er ekspert på norske skattelover og skal svare på spørsmålet: {query}\nBruk konteksten: {context}"
    #prompt = f"Du er en ekspert på norske skattelover og skal svare på følgende spørsmål: {question}. bruk følgende kontekst som kilde: {context}"
    
    messages = [
        ChatMessage(
            role="system", content=prompt
        )
    ]
    return messages

In [108]:
def vector_search_gpt(query):
    query_embedding = azure_openai_embedding_model.get_text_embedding(query)

    response_vector = es.search(
        index="index_skatteparagraf",
        query={
            "knn": {
                "field": "embedding",  # Ensure this matches the name of your dense_vector field
                "query_vector": query_embedding,
                "num_candidates": 50,  # This is optional but can help with performance/accuracy trade-offs
                "boost": 0.1,
            }
        },
        size=5,
        fields=["content"],
    )
    responses = [hit["_source"]["content"] for hit in response_vector["hits"]["hits"]]

    return responses

In [98]:
def ask_gpt(messages):
    answer = azure_openai_chat_model.chat(messages)
    print(answer)
    print("\n\n\n\n\n\n")
    print(prompt)

In [123]:
from llama_index.llms.ollama import Ollama

llm_ollama = Ollama(model="mistral", request_timeout=30.0)

messages = [
    ChatMessage(
        role="system", content='Du er ekspert på norske skattelover og skal svare dette spørsmålet på norsk: Hva er skattereglene når eier av arvegods dør og arvinger skal ta over?\nBruk konteksten: A-17-2.2.1 Overtar ansvaret for gjelden\nNår det bare er én arving etter avdøde og det ikke er gjenlevende ektefelle eller samboer og arvin -\ngen overtar fullt ansvar for avdødes gjeld, oppstår det ikke et dødsbo.\nI utgangspunktet skal enearving som har overtatt boet udelt skattlegges for sin og avdødes \nsamlede inntekt under ett i dødsåret, jf. sktl. §  2-15 første ledd. Betingelsene for skatteplikt må \nvurderes separat for henholdsvis avdøde og arving, se pkt. A-17-3. Arvingen kan fastsette inn -\ntektene hver for seg, men må formuesbeskattes for den samlede formue.\nSambeskatningen innebærer at arvingens underskudd for dødsåret og fremførbart underskudd \nfra tidligere år, kan avregnes mot avdødes overskudd i dødsåret.\nI tillegg kan avdødes underskudd fra tidligere år videreføres av enearvingen i samsvar med \nsktl. § 9-7 første ledd som gir enearvingen rett og plikt til å overta avdødes skatteposisjoner, se \nemnet A-16 Arv og gave – inngangsverdi og kontinuitet, pkt. A-16-2.\n\nE-1-9.12 Dødsfall\nAvdøde anses som eier fram til tidspunktet for dødsfallet.\nEt dødsbo er eier av formuesobjektene fra dødsfallet frem til disse utloddes. Etter utlodning \ner det den arvingen som overtar formuesobjektet som anses som eier. I tilfeller hvor skatt yter/\ntestamentarving er gjenstandslegatar, dvs. har krav på en bestemt gjenstand etter arvelater, blir \nimidlertid testamentsarvingen eier av gjenstanden ved arvefallet. Arvefallet vil normalt være \ntidspunktet for arvelaters død.\nEnearving som overtar boet udelt, anses som eier fra tidspunktet for dødsfallet. Det samme \ngjelder gjenlevende ektefelle/samboer i den utstrekning boet overtas i uskifte.\nHvis eiendommen overtas med begrensede rettigheter, må det foretas en helhetsvurdering av \nhvorvidt begrensningene er så vesentlige at eiendomsretten ikke kan anses for å være overført.'
    )
]

In [124]:
llm_ollama.chat(messages)

INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"


ChatResponse(message=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content=' Når en eier av arvegods dør og det kun er én arving, uten gjenlevende ægtemann eller samboer, og arven overtas fullstendig, oppstår det ikke et dødsbo. I dette tilfelle har arvingen skatteplikt for bothe sin egen og avdødes samlede inntekter i dødsåret (skattelov § 2-15 første ledd). Betingelsene for skatteplikt skal vurderes separat for avdøde og arving (A-17-3). Arvingen kan fastsette sine egne inntekter, men må formuesbeskattes for den samlede formuen.\n\nSamskattning betyr at arvingens underskudd for dødsåret og overskudd fra tidligere år kan avregnes mot avdødes overskudd i dødsåret. Desuden kan avdødes underskudd fra tidligere år videreføres av enearvingen i overensstemmelse med skattelov § 9-7 første ledd, som gir enearvingen rett og plikt til å overta avdødes skatteposisjoner (A-16 Arv og gave – inngangsverdi og kontinuitet, pt. A-16-2).\n\nEn død anses som eier av formuesobjektene frem til ut

In [114]:
question = "Hva er skattereglene når eier av arvegods dør og arvinger skal ta over?"
docs = vector_search_gpt(question)
prompt = prepare_prompt(question, docs)

INFO:httpx:HTTP Request: POST https://rag-poc-openai.openai.azure.com/openai/deployments/ada-002/embeddings?api-version=2023-12-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://rag-poc-openai.openai.azure.com/openai/deployments/ada-002/embeddings?api-version=2023-12-01-preview "HTTP/1.1 200 OK"
INFO:elastic_transport.transport:POST http://localhost:9200/index_skatteparagraf/_search [status:200 duration:0.017s]
POST http://localhost:9200/index_skatteparagraf/_search [status:200 duration:0.017s]


In [115]:
prompt

[ChatMessage(role=<MessageRole.SYSTEM: 'system'>, content='Du er ekspert på norske skattelover og skal svare på spørsmålet: Hva er skattereglene når eier av arvegods dør og arvinger skal ta over?\nBruk konteksten: A-17-2.2.1 Overtar ansvaret for gjelden\nNår det bare er én arving etter avdøde og det ikke er gjenlevende ektefelle eller samboer og arvin -\ngen overtar fullt ansvar for avdødes gjeld, oppstår det ikke et dødsbo.\nI utgangspunktet skal enearving som har overtatt boet udelt skattlegges for sin og avdødes \nsamlede inntekt under ett i dødsåret, jf. sktl. §  2-15 første ledd. Betingelsene for skatteplikt må \nvurderes separat for henholdsvis avdøde og arving, se pkt. A-17-3. Arvingen kan fastsette inn -\ntektene hver for seg, men må formuesbeskattes for den samlede formue.\nSambeskatningen innebærer at arvingens underskudd for dødsåret og fremførbart underskudd \nfra tidligere år, kan avregnes mot avdødes overskudd i dødsåret.\nI tillegg kan avdødes underskudd fra tidligere år

In [116]:
messages = [
    ChatMessage(
        role="system", content='Du er ekspert på norske skattelover og skal svare på spørsmålet: Hva er skattereglene når eier av arvegods dør og arvinger skal ta over?\nBruk konteksten: A-17-2.2.1 Overtar ansvaret for gjelden\nNår det bare er én arving etter avdøde og det ikke er gjenlevende ektefelle eller samboer og arvin -\ngen overtar fullt ansvar for avdødes gjeld, oppstår det ikke et dødsbo.\nI utgangspunktet skal enearving som har overtatt boet udelt skattlegges for sin og avdødes \nsamlede inntekt under ett i dødsåret, jf. sktl. §  2-15 første ledd. Betingelsene for skatteplikt må \nvurderes separat for henholdsvis avdøde og arving, se pkt. A-17-3. Arvingen kan fastsette inn -\ntektene hver for seg, men må formuesbeskattes for den samlede formue.\nSambeskatningen innebærer at arvingens underskudd for dødsåret og fremførbart underskudd \nfra tidligere år, kan avregnes mot avdødes overskudd i dødsåret.\nI tillegg kan avdødes underskudd fra tidligere år videreføres av enearvingen i samsvar med \nsktl. § 9-7 første ledd som gir enearvingen rett og plikt til å overta avdødes skatteposisjoner, se \nemnet A-16 Arv og gave – inngangsverdi og kontinuitet, pkt. A-16-2.\n\nE-1-9.12 Dødsfall\nAvdøde anses som eier fram til tidspunktet for dødsfallet.\nEt dødsbo er eier av formuesobjektene fra dødsfallet frem til disse utloddes. Etter utlodning \ner det den arvingen som overtar formuesobjektet som anses som eier. I tilfeller hvor skatt yter/\ntestamentarving er gjenstandslegatar, dvs. har krav på en bestemt gjenstand etter arvelater, blir \nimidlertid testamentsarvingen eier av gjenstanden ved arvefallet. Arvefallet vil normalt være \ntidspunktet for arvelaters død.\nEnearving som overtar boet udelt, anses som eier fra tidspunktet for dødsfallet. Det samme \ngjelder gjenlevende ektefelle/samboer i den utstrekning boet overtas i uskifte.\nHvis eiendommen overtas med begrensede rettigheter, må det foretas en helhetsvurdering av \nhvorvidt begrensningene er så vesentlige at eiendomsretten ikke kan anses for å være overført.'
    )
]

response = azure_openai_chat_model.chat(messages)
print(response)

INFO:openai._base_client:Retrying request to /deployments/gpt-4-turbo/chat/completions in 0.912239 seconds
Retrying request to /deployments/gpt-4-turbo/chat/completions in 0.912239 seconds
INFO:openai._base_client:Retrying request to /deployments/gpt-4-turbo/chat/completions in 1.772402 seconds
Retrying request to /deployments/gpt-4-turbo/chat/completions in 1.772402 seconds


KeyboardInterrupt: 

## NbBert pretrained model

In [29]:
from elasticsearch import Elasticsearch

def search(query, index="index_skatteparagraf", top_n=2):
    response = es.search(index=index, body={
        "size": top_n,
        "query": {
            "match": {
                "content": query  # Assuming your documents are stored under the "content" field
            }
        }
    })
    return [hit["_source"]["content"] for hit in response["hits"]["hits"]]


In [31]:
from transformers import MT5ForConditionalGeneration, T5Tokenizer

model_name = 'google/mt5-small'
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = MT5ForConditionalGeneration.from_pretrained(model_name)


In [44]:
prompt = "Summarize: What are the tax implications of working from home??"
input_ids = tokenizer.encode(prompt, return_tensors="pt", max_length=1024, truncation=True)

print(input_ids)
print("Input shape:", input_ids.shape)
print("Generated Output IDs:", output_ids)

output_ids = model.generate(
    input_ids,
    max_length=512,
    num_beams=5,
    no_repeat_ngram_size=2,
    temperature=0.9,
    top_k=50,
    top_p=0.9,
    early_stopping=True
)

tokenizer.decode(output_ids[0], skip_special_tokens=True)

tensor([[182651,  15472,    265,    267,   5126,    418,    287,  11577,    259,
            266, 102820,    304,  10357,    702,   2586,   1718,      1]])
Input shape: torch.Size([1, 17])
Generated Output IDs: tensor([[     0, 250099,      1]])


/Users/fabian.s.dietrichson/Library/Caches/pypoetry/virtualenvs/data-processing-A1QXdrwV-py3.12/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/Users/fabian.s.dietrichson/Library/Caches/pypoetry/virtualenvs/data-processing-A1QXdrwV-py3.12/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


'<extra_id_0>'

In [48]:
prompt = "summarize: What are the tax implications of working from home?"
input_ids = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024)
output_ids = model.generate(input_ids['input_ids'], max_length=512, num_beams=5, early_stopping=True)
answer = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print(answer)


<extra_id_0>


In [38]:
def generate_answer(question, documents):
    context = " ".join(documents)  # Combine documents into a single string
    prompt = f"Du er en ekspert på norske skattelover og skal svare på følgende spørsmål: {question}. bruk følgende kontekst som kilde: {context}"
    prompt = "What are the tax implications of working from home??"
    input_ids = tokenizer.encode(prompt, return_tensors="pt", max_length=1024, truncation=True)
    
    output_ids = model.generate(input_ids, max_length=512, num_beams=5, early_stopping=True)
    answer = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    
    return answer

In [30]:
# Example usage
question = "Hvordan fungerer eierforholdet til arving når eier dør?"
documents = search(question)  # Assume this fetches relevant documents from Elasticsearch
documents

INFO:elastic_transport.transport:POST http://localhost:9200/index_skatteparagraf/_search [status:200 duration:0.020s]
POST http://localhost:9200/index_skatteparagraf/_search [status:200 duration:0.020s]


['A-16-9.5.5 Medarvinger har rett til andel av salgsvederlag\nNår arving overtar fast eiendom på slike vilkår at medarvinger skal ha rett til andel i fremtidig \nsalgsvederlag for hele eller deler av eiendommen, skal den overtakende arving anses fullt ut som \neier av eiendommen, og anses å ha arvet en ideell andel på vanlig måte.',
 'S-10-3.4.7 Dødsfall\nLønn som er opptjent før skatt  yter dør, men som tidfestes etter dødsfallet og derfor skattlegges \nhos arving/gjenlevende ektefelle eller dødsbo, inngår ikke i beregningsgrunnlaget.']

In [39]:
generate_answer(question, documents)

'<extra_id_0>'

In [51]:
from transformers import pipeline
from transformers import AutoTokenizer
from transformers import AutoModel, BertLMHeadModel, AutoModelForQuestionAnswering

MODEL_NAME = "NbAiLab/nb-bert-base"
#pipe = pipeline("fill-mask", model=MODEL_NAME)
model = AutoModelForQuestionAnswering.from_pretrained("NbAiLab/nb-bert-base")

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at NbAiLab/nb-bert-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [35]:
context = "E-1-9.12 Dødsfall Avdøde anses som eier fram til tidspunktet for dødsfallet. Et dødsbo er eier av formuesobjektene fra dødsfallet frem til disse utloddes. Etter utlodning er det den arvingen som overtar formuesobjektet som anses som eier. I tilfeller hvor skatt yter/ testamentarving er gjenstandslegatar, dvs. har krav på en bestemt gjenstand etter arvelater, blir imidlertid testamentsarvingen eier av gjenstanden ved arvefallet. Arvefallet vil normalt være tidspunktet for arvelaters død. Enearving som overtar boet udelt, anses som eier fra tidspunktet for dødsfallet. Det samme gjelder gjenlevende ektefelle/samboer i den utstrekning boet overtas i uskifte. Hvis eiendommen overtas med begrensede rettigheter, må det foretas en helhetsvurdering av hvorvidt begrensningene er så vesentlige at eiendomsretten ikke kan anses for å være overført."

In [36]:
question = "Hvordan fungerer eierforholdet til arving når eier dør?"

In [54]:
# Define the maximum length
max_length = 512  # or higher if the model supports it

# Encode the inputs, specifying the max_length
inputs = tokenizer.encode_plus(
    question,
    context,
    max_length=max_length,
    truncation="only_second",
    return_tensors='pt',
    add_special_tokens=True
)


In [56]:
import torch

# Get model predictions
with torch.no_grad():
    outputs = model(**inputs)

# Get start and end scores for answer
answer_start_scores, answer_end_scores = outputs.start_logits, outputs.end_logits
answer_start = torch.argmax(answer_start_scores)
answer_end = torch.argmax(answer_end_scores) + 1

# Convert tokens to answer
answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][answer_start:answer_end]))

print("Answer:", answer)


Answer: overtar formuesobjektet som anses som eier. I tilfeller hvor skatt yter / testamentarving er gjenstandslegatar, dvs. har krav på en bestemt gjenstand etter arvelater, blir imidlertid testamentsarvingen eier av gjenstanden ved arvefallet. Arve


In [41]:
prompt = tokenizer(
    question,
    context,
    max_length=384,
    truncation="only_second",
    return_offsets_mapping=True,
    padding="max_length",
)

In [50]:
output = model.generate(prompt)

AttributeError: 